# Week3_4 Assignment

## [BASIC](#Basic) 
- Encoder & Decoder Layer 코드를 직접 필사하고 각 함수에 주석을 달 수 있다. 

## [CHALLENGE](#Challenge)
- 텐서의 크기(shape)를 계산할 수 있다. 

## [ADVANCED](#Advanced)
- 완성된 transformer 모델의 모든 학습 가능한 파라미터 이름과 크기(shape)를 출력할 수 있다.

### Informs
이번 과제에서는 "[Annotated Transformer](https://nlp.seas.harvard.edu/2018/04/03/attention.html)"의 코드를 필사해본다.   
"Annotated Transformer"는 "Attention is all you need" 논문에서 제안한 transformer 모델을 pytorch 라이브러리로 직접 구현한다.   
코드 필사를 통해 다음을 배울 수 있다.    
- Encoder, Decoder 구조
- Attention Mechanism
- "residual connection", "layer normalization" 등의 구조 

코드 필사를 시작하기 앞서, transformer 모델의 최종 구조를 살펴보자.    

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_full.png?raw=true" width="500" align="center"/>

최종 모델은 `EncoderDecoder()` 클래스에 여러 인스턴스를 생성자의 입력 파라미터로 넣어 생성한다.    
앞으로 우리는 `EncoderDecoder()` 클래스와 같은 여러 클래스들을 구현하고 연결할 것이다. 따라서 대략적인 클래스간의 관계를 살펴보고 이해한다면 보다 큰 그림을 가지고 코드 필사를 할 수 있을 것이다. 

Transformer 모델은 크게 4가지 클래스로 구현된다.    
- Frame
    - frame 역할을 하는 `EncoderDecoder` 클래스
- Input Embedding & Encoding
    - 입력값을 벡터화하는 `Embeddings`, `PositionalEncoding`
- Encoder & Decoder
    - 각 6개 layer를 갖고 있는 `Encoder`, `Decoder`
    - layer 1층을 구현한 `EncoderLayer`, `DecoderLayer`
- Sublayer
    - `EncoderLayer`, `DecoderLayer` 내부에서 사용되는 Sublayer 클래스인 `MultiHeadAttiontion`, `PositionwiseFeedForward`
    - Sublayer 클래스들을 연결하는 `SublayerConnection`
    
아래 좌측 도식에서 각 클래스의 색상은 아래 우측 도식(transformer 구조)의 색상과 맵핑되어 있다.    
각 클래스의 역할과 클래스 간 연결 관계를 생각하면서 transformer를 코드로 구현해보자.   


<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_map.png?raw=true" width="400" height="400" align="left"/>
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_transformer.png?raw=true" width="300" height="400" align="right"/>



In [ ]:
import os
import sys
import pandas as pd
import numpy as np 

import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable

import math, copy, time
import random

## Basic

### Frame
- `EncoderDecoder`

아래 도식은 `EncoderDecoder` 클래스의 `forward()`, `encode()`, `decode()` 메소드를 도식화 한 것이다.    
 
<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_encoderdecoder.png?raw=true" width=500>


- `Generator`

In [ ]:
class EncoderDecoder(nn.Module):
    #기본 encoder-decoder 구조 작성
    def __init__(self, encoder, decoder, src_embed, tgt_embed, generator):
        super(EncoderDecoder, self).__init__()
        self.encoder = encoder
        self.decoder = decoder
        self.src_embed = src_embed # 인코더의 임베딩-위치인코딩
        self.tgt_embed = tgt_embed # 디코더의 임베딩-위치인코딩
        self.generator = generator
    
    #Auto-regressive를 가능하게 해주는 함수(이전 단어의 output을 다음 input으로 사용)
    def forward(self, src, tgt, src_mask, tgt_mask):
      return self.decode(self.decode(src, src_mask), src_mask, tgt, tgt_mask)
    
    #encoder에 입력하는 함수
    def encode(self, src, src_mask):
      return self.encoder(self.src_embed(src), src_mask)
    
    #decoder에 입력하는 함수
    def decode(self, memory, src_mask, tgt, tgt_mask):
      return self.decoder(self.tgt_embed(tgt), memory, src_mask, tgt_mask)

In [ ]:
class Generator(nn.Module):
    #linear generator
    def __init__(self, d_model, vocab):
      super(Generator, self).__init__()
      self.proj = nn.Linear(d_model, vocab)
    
    #softmax 적용함수
    def forward(self, x):
      return F.log_softmax(self.proj(x), dim=-1)

### Encoder
- `Encoder`
- `EncoderLayer`
- `SublayerConnection`
- Reference
    - Layer Normalization
        - [한국어 설명](https://yonghyuc.wordpress.com/2020/03/04/batch-norm-vs-layer-norm/)
        - [torch official docs](https://pytorch.org/docs/stable/generated/torch.nn.LayerNorm.html)
    - Residual Connection
        - [한국어 설명](https://itrepo.tistory.com/36)
    - pytorch ModuleList
        - [torch official docs](https://pytorch.org/docs/1.9.1/generated/torch.nn.ModuleList.html)


In [ ]:
# N개의 module을 복제
def clones(module, N):
  return nn.ModuleList([copy.deepcopy(module) for _ in range(N)])

In [ ]:
class Encoder(nn.Module):
    #클래스 변수 정의
    def __init__(self, layer, N):
      super(Encoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)
    
    #생성된 layer에 input이 통과하도록 진행
    def forward(self, x, mask):
      for layer in self.layers:
        x = layer(x, mask)
      return self.norm(x)

In [ ]:
class LayerNorm(nn.Module):
    #features 크기만큼 1, 0으로 이루어진 Parameter 생성 
    def __init__(self, features, eps=1e-6):
      super(LayerNorm, self).__init__()
      self.a_2 = nn.Parameter(torch.ones(features)) #weight
      self.b_2 = nn.Parameter(torch.zeros(features)) #bias
      self.eps = eps
    
    # (a - 데이터별 평균(a)) / root(데이터별 분산(a)+epsilon) + bias
    def forward(self, x):
      mean = x.mean(-1, keepdim=True)
      std = x.std(-1, keepdim=True)
      return self.a_2 * (x - mean) / (std + self.eps) + self.b_2

In [ ]:
class SublayerConnection(nn.Module):
    #Sublayer를 연결해주는 클래스
    #Connection을 위해 sub-layer,embedding layer, output 모두 dimention = 512로 동일.
    def __init__(self, size, dropout):
      super(SublayerConnection, self).__init__()
      self.norm = LayerNorm(size)
      self.dropout = nn.Dropout(dropout)

    #add and Norm에는 Normalize 와 dropout이 적용됨.
    def forward(self, x, sublayer):
      return x + self.dropout(sublayer(self.norm(x)))

In [ ]:
#Encoder에는 2개의 sub-layer가 존재 = Multi-Head self Attention + FFN
class EncoderLayer(nn.Module):
    
    def __init__(self, size, self_attn, feed_forward, dropout):
      super(EncoderLayer, self).__init__()
      self.self_attn = self_attn
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 2)
      self.size = size

    #첫번째 sub-layer connection에 X개의 self-attention을 연결
    #두번째 sub-layer conneciton에 FFN 연결
    def forward(self, x, mask):
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, mask))
      return self.sublayer[1](x, self.feed_forward)

### Decoder
- `Decoder`
- `DecoderLayer`

In [ ]:
class Decoder(nn.Module):
    #Decoder layer를 N개로 복제
    def __init__(self, layer, N):
      super(Decoder, self).__init__()
      self.layers = clones(layer, N)
      self.norm = LayerNorm(layer.size)
    
    # N개의 decoder layer 마다 Decode 실행 후 normalization
    def forward(self, x, memory, src_mask, tgt_mask):
      for layer in self.layers:
        x = layer(x, memory, src_mask, tgt_mask)
      return self.norm(x)

In [ ]:
class DecoderLayer(nn.Module):

    def __init__(self, size, self_attn, src_attn, feed_forward, dropout):
      # self_attn: MultiHeadedAttention
      # src_attn: MultiHeadedAttention
      
      super(DecoderLayer, self).__init__()
      self.size = size
      self.self_attn = self_attn
      self.src_attn = src_attn # cross attention
      self.feed_forward = feed_forward
      self.sublayer = clones(SublayerConnection(size, dropout), 3)
    
    #첫번째 sub-layer에 masked self_attention 적용
    #두번째 sub-layer에 cross self_attention 적용
    #세번째 sub-layer에 FFN 적용
    def forward(self, x, memory, src_mask, tgt_mask):
      m = memory #인코더에서 넘어온 인코딩
      x = self.sublayer[0](x, lambda x: self.self_attn(x, x, x, tgt_mask))
      x = self.sublayer[1](x, lambda x: self.src_attn(x, m, m, src_mask))
      return self.sublayer[2](x, self.feed_forward)

### Sublayer
- `attention` 함수

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_attention.png?raw=true" width="500" align="center"/>  

- `MultiHeadedAttention`
- `PositionwiseFeedForward`

### Challenge


### Q1. 위 도식에 따라 `score`, `p_attn`, `attention` 을 구하라 

In [ ]:
def attention(query, key, value, mask=None, dropout=None):
  d_k = query.size(-1)
  scores = torch.matmul(query, key.transpose(-2, -1)) \
            / math.sqrt(d_k)
  if mask is not None:
      scores = scores.masked_fill(mask == 0, -1e9)
  p_attn = F.softmax(scores, dim = -1)
  if dropout is not None:
      p_attn = dropout(p_attn)
  return torch.matmul(p_attn, value), p_attn

In [ ]:
#Q2
key = [[0.1,0.2],[0.3,0.4],[0.5,0.6],[0.7,0.8]]
query = [[0.1,0.2],[0.3,0.4],[0.5,0.6],[0.7,0.8]]
value = [[0.1,0.2],[0.3,0.4],[0.5,0.6],[0.7,0.8]]
key = torch.tensor(key)
query = torch.tensor(query)
value = torch.tensor(value)
p_attn,attention = attention(query,key,value)

In [ ]:
print(score)
print(p_attn)
print(attention)

tensor([[0.0354, 0.0778, 0.1202, 0.1626],
        [0.0778, 0.1768, 0.2758, 0.3748],
        [0.1202, 0.2758, 0.4313, 0.5869],
        [0.1626, 0.3748, 0.5869, 0.7990]])
tensor([[0.2343, 0.2445, 0.2551, 0.2661],
        [0.2142, 0.2365, 0.2611, 0.2883],
        [0.1950, 0.2278, 0.2662, 0.3110],
        [0.1769, 0.2186, 0.2703, 0.3342]])
tensor([[0.4106, 0.5106],
        [0.4247, 0.5247],
        [0.4386, 0.5386],
        [0.4524, 0.5524]])


In [ ]:
#Q4
a = nn.Linear(512,512)
input = torch.randn(12,512)
query = a(input).view(12,-1,8,64).transpose(1,2)
key = a(input).view(12,-1,8,64).transpose(1,2)
value = a(input).view(12,-1,8,64).transpose(1,2)
score,p_attn,attention = attention(query,key,value)

In [ ]:
print(score.size())
print(p_attn.size())
print(attention.size())

torch.Size([12, 8, 1, 1])
torch.Size([12, 8, 1, 64])
torch.Size([12, 8, 1, 1])


###Q2. query, key, value가 모두 (m, d_k) shape의 matrix라고 가정할 때, `score`, `p_attn`, `attention`의 shape을 각각 구하라
- score : (m,m)
- p_attn : (m,m)
- attention : (m,d_k)

### (아래의 **Q3을 먼저 풀고 돌아오세요**) Q4.  query, key, value가 모두 (12, 8, 1, 64) shape의 tensor라고 가정할 때 , `score`, `p_attn`, `attention`의 shape을 각각 구하라

- score : (12,8,1,1)
- p_attn : (12,8,1,64)
- attention : (12,8,1,1)

- `MultiHeadedAttention`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_multihead.png?raw=true" width="300" align="center"/>  

In [ ]:
class MultiHeadedAttention(nn.Module):
    def __init__(self, h, d_model, dropout=0.1):
      super(MultiHeadedAttention, self).__init__()
      assert d_model % h == 0
      # We assume d_v always equals d_k
      self.d_k = d_model // h   # Q, K의 차원
      self.h = h
      self.linears = clones(nn.Linear(d_model, d_model), 4)
      self.attn = None
      self.dropout = nn.Dropout(p=dropout)
      #Wq, Wk, Wv = 1/h개
      # weight를 여러개 하지 않고 1개의 weight로 h개의 scaled dot-product attention 공유
    
    def forward(self, query, key, value, mask=None):
      if mask is not None:
          # Same mask applied to all h heads.
          mask = mask.unsqueeze(1)
      nbatches = query.size(0)
      
      # 1) Do all the linear projections in batch from d_model => h x d_k 
      query, key, value = \
          [l(x).view(nbatches, -1, self.h, self.d_k).transpose(1, 2)
            for l, x in zip(self.linears, (query, key, value))]
      #self.h*self.d_k = d_model
      
      # 2) Apply attention on all the projected vectors in batch. 
      x, self.attn = attention(query, key, value, mask=mask, 
                                dropout=self.dropout)
      
      # 3) "Concat" using a view and apply a final linear. 
      x = x.transpose(1, 2).contiguous() \
            .view(nbatches, -1, self.h * self.d_k)
      return self.linears[-1](x)
            

In [ ]:
#Q3
a = nn.Linear(512,512)
input = torch.randn(12,512)
output1 = a(input).view(12,-1,8,64)
output2 = a(input).view(12,-1,8,64).transpose(1,2)

print(output1.size())
print(output2.size())

torch.Size([12, 1, 8, 64])
torch.Size([12, 8, 1, 64])


### Q3.  query, key, value가 모두 (12, 512) shape의 matrix이고, h 값이 8 이라고 가정할 때, 아래 값의 shape을 각각 구하라

- `d_k` (d_k = d_model // h) : 512/8 = 64
- `nn.Linear(d_model, d_model)(query)` : (12,512)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k)` : (12, 1 ,8, 64)
- `nn.Linear(d_model, d_model)(query).view(nbatches, -1, h, d_k).transpose(1,2)` : (12, 8, 1, 64)

- `PositionwiseFeedForward`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pwff.png?raw=true" width="300" align="center"/>  

In [ ]:
class PositionwiseFeedForward(nn.Module):
  def __init__(self, d_model, d_ff, dropout=0.1):
    super(PositionwiseFeedForward, self).__init__()
    #위치정보를 d_model에서 d_ff로 d_ff 에서 d_model로 변환
    self.w_1 = nn.Linear(d_model, d_ff)
    self.w_2 = nn.Linear(d_ff, d_model)
    self.dropout = nn.Dropout(dropout)

  def forward(self, x):
    return self.w_2(self.dropout(F.relu(self.w_1(x))))

### Input Embedding & Encoding
- `Embeddings`
    - [pytorch official docs](https://pytorch.org/docs/stable/generated/torch.nn.Embedding.html)

In [ ]:
class Embeddings(nn.Module):

    #d_model의 차원의 벡터로 변환
    def __init__(self, d_model, vocab):
      super(Embeddings, self).__init__()
      self.lut = nn.Embedding(vocab, d_model)
      self.d_model = d_model
    
    #임베딩 벡터에 sqrt(d_model)을 곱해서 임베딩 벡터의 요소들 값을 증가시킨다.
    #곱하는 이유, 뒤에 더해질 포지션 벡터에 의해 임베딩 벡터의 결과가 희석되지 않도록 하기위함.
    def forward(self, x):
      return self.lut(x) * math.sqrt(self.d_model)

- `PositionalEncoding`

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_pe.png?raw=true" width="500" align="center"/>  

- `position` 변수 설명
    - 모든 position (=최대 토큰 개수)의 값을 갖고 있는 matrix
- `div_term` 변수 설명

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_div.png?raw=true" width="500" align="center"/>  
- `Embedding` + `Encoding` 도식화 

<img src="https://github.com/ChristinaROK/PreOnboarding_AI_assets/blob/36a670a7b6233d5218a495150beb337a899ecb70/week3/week3_3_emb_enc.png?raw=true" width="400" align="center"/>  


In [ ]:
class PositionalEncoding(nn.Module):
    
    def __init__(self, d_model, dropout, max_len = 5000):
      super(PositionalEncoding, self).__init__()
      self.dropout = nn.Dropout(p=dropout)
      
      # Compute the positional encodings once in log space.
      pe = torch.zeros(max_len, d_model)
      position = torch.arange(0, max_len).unsqueeze(1)
      div_term = torch.exp(torch.arange(0, d_model, 2) *
                            -(math.log(10000.0) / d_model))
      pe[:, 0::2] = torch.sin(position * div_term)
      pe[:, 1::2] = torch.cos(position * div_term)
      pe = pe.unsqueeze(0)
      self.register_buffer('pe', pe)
        
    def forward(self, x):
      x = x + Variable(self.pe[:, :x.size(1)], 
                        requires_grad=False)
      return self.dropout(x)

In [ ]:
#Q4
max_len=512
d_model=512

pe = torch.zeros(max_len, d_model)
position = torch.arange(0, max_len).unsqueeze(1)
print(position.size())
div_term = torch.exp(torch.arange(0, d_model, 2) *
                      -(math.log(10000.0) / d_model))
print(div_term.size())
print((position * div_term).size())
pe[:, 0::2] = torch.sin(position * div_term)
pe[:, 1::2] = torch.cos(position * div_term)
pe = pe.unsqueeze(0)

torch.Size([512, 1])
torch.Size([256])
torch.Size([512, 256])


### Q4.  max_len이 512이고, d_model이 512라고 가정할 때, `position`과 `div_term`의 shape을 구하라

- `position` : (512,1)
- `div_term` : (256)
- `position * div_term` : (512,256) 

### Advanced

### Finally Build Model
- Xavier Initialization
    - [한국어 자료](https://huangdi.tistory.com/8)
    - [pytorch official docs](https://pytorch.org/docs/stable/nn.init.html#torch.nn.init.xavier_uniform_)

In [ ]:
def make_model(src_vocab, tgt_vocab, 
               N=6, d_model=512, d_ff=2048, h=8, dropout=0.1):
  c = copy.deepcopy
  attn = MultiHeadedAttention(h, d_model)
  ff = PositionwiseFeedForward(d_model, d_ff, dropout)
  position = PositionalEncoding(d_model, dropout)
  model = EncoderDecoder(
      Encoder(EncoderLayer(d_model, c(attn), c(ff), dropout), N),
      Decoder(DecoderLayer(d_model, c(attn), c(attn), 
                            c(ff), dropout), N),
      nn.Sequential(Embeddings(d_model, src_vocab), c(position)),
      nn.Sequential(Embeddings(d_model, tgt_vocab), c(position)),
      Generator(d_model, tgt_vocab))
  
  # This was important from their code. 
  # Initialize parameters with Glorot / fan_avg.
  for p in model.parameters():
      if p.dim() > 1:
          nn.init.xavier_uniform(p)
  return model
            

In [ ]:
model = make_model(10,10)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:19: UserWarning: nn.init.xavier_uniform is now deprecated in favor of nn.init.xavier_uniform_.


### Q5. 위 코드로 만든 모델의 모든 파라미터의 이름과 크기 (shape) 을 출력하라

In [ ]:
# 구현
for name, param in model.named_parameters():
  print(param.size())
  print(name)

torch.Size([512, 512])
encoder.layers.0.self_attn.linears.0.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.0.bias
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.1.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.1.bias
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.2.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.2.bias
torch.Size([512, 512])
encoder.layers.0.self_attn.linears.3.weight
torch.Size([512])
encoder.layers.0.self_attn.linears.3.bias
torch.Size([2048, 512])
encoder.layers.0.feed_forward.w_1.weight
torch.Size([2048])
encoder.layers.0.feed_forward.w_1.bias
torch.Size([512, 2048])
encoder.layers.0.feed_forward.w_2.weight
torch.Size([512])
encoder.layers.0.feed_forward.w_2.bias
torch.Size([512])
encoder.layers.0.sublayer.0.norm.a_2
torch.Size([512])
encoder.layers.0.sublayer.0.norm.b_2
torch.Size([512])
encoder.layers.0.sublayer.1.norm.a_2
torch.Size([512])
encoder.layers.0.sublayer.1.norm.b_2
torch.Size([512, 512])
enco